## Homework 08:  Parallel Programming 01

## Due Date: Apr 20, 2020, 08:00am

#### Firstname Lastname: 

#### E-mail: 

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

Consequently, for $r=0, 1, \dots, N - 1$, Process $r$ squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ will print the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$.



**Note**: You can use either blocking or non-blocking operations. Any input read from the user must be validated correctly. Make sure to provide adequate comments and documentation in the code. 



### Solution: Problem 1

In [1]:
%%writefile 2020_spring_sol08_pr01.py

import sys

import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

var_buffer = np.array([0], dtype=np.int64)

if rank == 0:
    comm.Send(var_buffer, dest = 1 % size)
    print("Process", rank, "sent", var_buffer, "to Process", (rank + 1) % size)
    comm.Recv(var_buffer, source=size - 1)
    print("Process", rank, "received", var_buffer, "from Process", (rank - 1) % size)
    print("Process", rank, "final evaluation:", var_buffer)
    
for r in range(1, size):
    if rank == r:
        comm.Recv(var_buffer, source=r - 1)
        print("Process", rank, "received", var_buffer, "from Process", (r - 1) % size)
        var_buffer += r ** 2
        comm.Send(var_buffer, dest = (r + 1) % size)
        print("Process", rank, "sent", var_buffer, "to Process", (r + 1) % size)

        

Overwriting 2020_spring_sol08_pr01.py


The output will be the sum of the squares of the first $N - 1$ natural numbers

\begin{equation}
\sum_{r = 1}^{N-1} r^2 = (N - 1) N (2 N - 1) / 6,
\end{equation}


which is equal to $285, 1015, 2470, 4900$, evaluated at $N=10, 15, 20, 25$, respectively.


---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations. Any input read from the user must be validated correctly. Make sure to provide adequate comments and documentation in the code.



### Solution: Problem 2

In [2]:
%%writefile 2020_spring_sol08_pr02.py

import sys

import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

T = 10

# var_buffer = [x, y, p, message_counter]
var_buffer = np.array([1, 1, 1, 0], dtype=np.float64)

# Read x and y and Message00
if var_buffer[-1] == 0 and rank == 0:
    for line in sys.stdin:
        x = float(line)        
        if x != 0.0:
            break
    for line in sys.stdin:
        y = float(line)        
        if y != 0.0:
            break

    print("x, y:", x, y)

    var_buffer[0] = x
    var_buffer[1] = y
    var_buffer[2] = x
    # increase the  message counter
    var_buffer[-1] += 1

    comm.Send(var_buffer, dest=1)
    print("Msg 0", "Process 0 sent: ", var_buffer, "to Process 1")
    

for i in range(1, T):
    if rank == i % 2:
        comm.Recv(var_buffer, source = (i + 1) % 2)
        print("Msg", i, " Process", i % 2, "received", var_buffer, "from Process", (i + 1) % 2)
        
        if rank == 0:
            var_buffer[2] *= var_buffer[0]
        else:
            var_buffer[2] /= var_buffer[1]
        # increase the  message counter
        var_buffer[-1] += 1

        comm.Send(var_buffer, dest = (i + 1) % 2)
        print("Msg", i, " Process", i % 2, "sent", var_buffer, "to Process", (i + 1) % 2)
        if i == T - 1:
            print("Msg", i, " Process", i % 2, "Final Value:", var_buffer[2])

        


Overwriting 2020_spring_sol08_pr02.py


The results for: $(x, y) = (2, 4), (1, 3), (5, 7), (5, 10)$ are: $(1/2)^ 5, (1/3)^5, (5/7)^5, (1/2)^5$ respectively. 

In fact, the result is $(x/y) ^ {N/2}$ given $N$ is divisible by $2$.
